### Preprocessing e impostazione della rete

#### Inserimento dei pesi e rimozione colonna "type"

In [5]:
import pandas as pd

# Caricare i file CSV
characters_df = pd.read_csv('Data/characters.csv')  # Carica il file CSV dei personaggi
relations_df = pd.read_csv('Data/relations.csv')  # Carica il file CSV delle relazioni

# Parole chiave per le relazioni familiari
family_keywords = ['brother', 'sister', 'father', 'mother', 'son', 'daughter', 'cousin']

# Creiamo un dizionario per le biografie dei personaggi
character_bio = characters_df.set_index('id')['bio'].to_dict()

# Funzione per classificare le relazioni
def classify_relation(row):
    source_bio = character_bio.get(row['source'], "").lower()
    target_bio = character_bio.get(row['target'], "").lower()
    
    # Verifica se ci sono relazioni familiari
    if any(keyword in source_bio for keyword in family_keywords) or any(keyword in target_bio for keyword in family_keywords):
        return 'family'
    # Altri tipi di relazione
    elif row['type'] == '+':
        return 'friendship'  # Amicizia
    elif row['type'] == '-':
        return 'enmity'  # Nemici
    return 'neutral'  # Relazione neutrale

# Applicare la funzione di classificazione delle relazioni
relations_df['relation_detail'] = relations_df.apply(classify_relation, axis=1)

# Mappare i pesi per ogni tipo di relazione
weights = {
    'family': 3,
    'friendship': 2,
    'alliance': 1,
    'enmity': -3,
    'neutral': 0,
}
relations_df['weight'] = relations_df['relation_detail'].map(weights)

# Rimuovere la colonna "type"
relations_df = relations_df.drop(columns=['type'])

# Salvare il DataFrame aggiornato in un nuovo file CSV
relations_df.to_csv('Data/relations_with_details.csv', index=False)

# Visualizzare i primi risultati
print(relations_df.head())

   source  target relation_detail  weight
0       0       1          family       3
1       0      25          family       3
2       0      45          family       3
3       1       0          family       3
4       1      11          family       3


#### Unione dei dataset

Vedere se è utile avere uno unico (alcuni dati si ripetono)

In [6]:
import pandas as pd

# Carica i file CSV
relations_df = pd.read_csv('Data/relations_with_details.csv')  # Relazioni con dettagli
characters_df = pd.read_csv('Data/characters.csv')  # Personaggi

# Unire le informazioni sui personaggi per il 'source' (personaggio di partenza)
relations_df = relations_df.merge(characters_df[['id', 'name', 'bio']], how='left', left_on='source', right_on='id')
relations_df = relations_df.rename(columns={'name': 'source_name', 'bio': 'source_bio'})

# Unire le informazioni sui personaggi per il 'target' (personaggio di arrivo)
relations_df = relations_df.merge(characters_df[['id', 'name', 'bio']], how='left', left_on='target', right_on='id')
relations_df = relations_df.rename(columns={'name': 'target_name', 'bio': 'target_bio'})

# Rimuovere le colonne temporanee 'id' usate per il merge
relations_df = relations_df.drop(columns=['id_x', 'id_y'])

# Salvare il DataFrame combinato in un nuovo file CSV
relations_df.to_csv('Data/combined_relations.csv', index=False)

# Visualizzare i primi risultati
print(relations_df.head())

   source  target relation_detail  weight             source_name  \
0       0       1          family       3  Regulus Arcturus Black   
1       0      25          family       3  Regulus Arcturus Black   
2       0      45          family       3  Regulus Arcturus Black   
3       1       0          family       3            Sirius Black   
4       1      11          family       3            Sirius Black   

                                          source_bio             target_name  \
0  Brother of Sirius. Used to be a Death Eater bu...            Sirius Black   
1  Brother of Sirius. Used to be a Death Eater bu...     Bellatrix Lestrange   
2  Brother of Sirius. Used to be a Death Eater bu...          Lord Voldemort   
3  Best friend of James Potter and godfather of H...  Regulus Arcturus Black   
4  Best friend of James Potter and godfather of H...        Albus Dumbledore   

                                          target_bio  
0  Best friend of James Potter and godfather of H